<a href="https://colab.research.google.com/github/Griznt/colab_test_tasks/blob/main/alphabank_test_task_akramovmv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Задание 1
Приведите в качестве решения скриншот с результатом, скрипт, а также свои комментарии относительно предложенного решения (с указанием среды, в которой решали задания).


##SQL1
Отобрать клиентов по г. Москва с суммарными остатками по клиенту от 20 000 на последнюю дату.


Обновим версию SQLite3 до более новой, с поддержкой оконных функций 
(будет перезапуск session)

In [ ]:
!gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
!mv _sqlite3.cpython-37m-x86_64-linux-gnu.so /usr/lib/python3.7/lib-dynload/
import os
os.kill(os.getpid(), 9)

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi
To: /content/_sqlite3.cpython-37m-x86_64-linux-gnu.so
100% 6.50M/6.50M [00:00<00:00, 81.9MB/s]


In [ ]:
import sqlite3
import pandas as pd

In [ ]:
sqlite3.sqlite_version

'3.38.0'

In [ ]:
con = sqlite3.connect('db')

In [ ]:
def select(sql):
  return pd.read_sql(sql, con)

Заполним БД тестовыми данными.

In [ ]:
cur = con.cursor()

In [ ]:
sql = '''
drop table if exists clients;

create table clients (client_id varchar(1), fio varchar (255) , region varchar(50), account_num int);

insert into clients values  
('A','Иванов','Москва',111), 
('A','Иванов','Москва',222), 
('B','Петров','Иваново',333), 
('C','Сидоров','Москва',444) 

'''

In [ ]:
cur.executescript(sql)

In [ ]:
sql = '''select t.* from clients t'''

In [ ]:
select(sql)

,client_id,fio,region,account_num
0,A,Иванов,Москва,111
1,A,Иванов,Москва,222
2,B,Петров,Иваново,333
3,C,Сидоров,Москва,444


In [ ]:
sql = '''
drop table if exists accounts;

create table accounts ([date] date, summa_usd money, account_num int);

insert into accounts values 
('2012-01-01',15000,111), 
('2012-02-01',10000,111), 
('2012-02-01',5000,222), 
('2012-03-01',30000,333), 
('2012-04-01',20000,444)

'''

In [ ]:
cur.executescript(sql)

In [ ]:
sql = '''select t.* from accounts t'''

In [ ]:
select(sql)

,date,summa_usd,account_num
0,2012-01-01,15000,111
1,2012-02-01,10000,111
2,2012-02-01,5000,222
3,2012-03-01,30000,333
4,2012-04-01,20000,444


Начнем анализ, в соотствествии с заданием:
> Отобрать клиентов по г. Москва с суммарными остатками по клиенту от 20 000 на последнюю дату. 

В текущем решении буду учесть для каждого клиента его денежные средства на последнюю дату для этого клиента по текущему счету


In [ ]:
sql = '''select * from accounts'''

In [ ]:
select(sql)

,date,summa_usd,account_num
0,2012-01-01,15000,111
1,2012-02-01,10000,111
2,2012-02-01,5000,222
3,2012-03-01,30000,333
4,2012-04-01,20000,444


####Первый вариант решения

Найдем максимальную дату для каждого аккаунта (account_num)

In [ ]:
sql = '''
select 
  a.account_num,
  max(a.date) as max_date
from accounts a
group by 1
'''

In [ ]:
select(sql)

,account_num,max_date
0,111,2012-02-01
1,222,2012-02-01
2,333,2012-03-01
3,444,2012-04-01


Получили последнюю дату для каждого account_num.

Сейчас отфильтруем данные в таблице accounts по полученным данным с max_date

In [ ]:
sql = '''
with account_max_date as (
  select 
    a.account_num,
    max(a.date) as max_date
  from accounts a
  group by 1
)


select a.* from accounts a
  inner join account_max_date ald 
    on a.account_num = ald.account_num
    and a.date = ald.max_date 
'''

In [ ]:
select(sql)

,date,summa_usd,account_num
0,2012-02-01,10000,111
1,2012-02-01,5000,222
2,2012-03-01,30000,333
3,2012-04-01,20000,444


Далее, с помощью `inner join` отфильтруем и оставим только такие записи, где для каждого account_num возьмем максимальную дату.

In [ ]:
sql = '''
with account_max_date as (
  select 
    a.account_num,
    max(a.date) as max_date
  from accounts a
  group by 1
),

accounts_filtered as (
  select a.* from accounts a
    inner join account_max_date ald 
      on a.account_num = ald.account_num
      and a.date = ald.max_date 
)

select 
  c.client_id,
  c.fio,
  c.region,
  af.date,
  sum(af.summa_usd) as total_sum
from clients c
  left join accounts_filtered af on c.account_num = af.account_num
group by 1
having c.region = 'Москва'
  and  total_sum >= 20000
'''

In [ ]:
select(sql)

,client_id,fio,region,date,total_sum
0,C,Сидоров,Москва,2012-04-01,20000


####Второе решение, правильное:

In [ ]:
sql = '''
  with accounts_rank as (
    select 
      t.*, dense_rank() over (partition by t.account_num order by t.date desc) as rnk
    from accounts t),
    
  accounts_filter as (
    select t.* from accounts_rank t
    where t.rnk = 1)

  select t.client_id, t.fio, t.region, sum(a.summa_usd) as summa_usd_sum 
  from clients t
  join accounts_filter a
  on t.account_num = a.account_num
  group by t.client_id, t.fio, t.region
  having t.region like 'Москва' and summa_usd_sum >= 20000


'''

In [ ]:
select(sql)

,client_id,fio,region,summa_usd_sum
0,C,Сидоров,Москва,20000


Отлично! 

### Ответ
Сидоров

##SQL2

Таблица #oper хранит информацию о количестве произведенных операций на каждую календарную дату.  
Вывести на каждую дату количество операций, совершенное с начала месяца по указанную дату включительно накопительным итогом.


Создадим таблицу.

Не понятно, какой формат даты, укажем явно: `'%Y-%m-%d`

In [ ]:
sql = '''
drop table if exists oper;

create table oper (date date, cnt integer);

insert into oper values 
(strftime('%Y-%m-%d', '2019-06-02'),     1985), 
(strftime('%Y-%m-%d', '2019-06-03'),     1577), 
(strftime('%Y-%m-%d', '2019-06-04'),     1597), 
(strftime('%Y-%m-%d', '2019-06-05'),     1468), 
(strftime('%Y-%m-%d', '2019-07-06'),     82), 
(strftime('%Y-%m-%d', '2019-07-08'),    1689), 
(strftime('%Y-%m-%d', '2019-07-09'),     1556), 
(strftime('%Y-%m-%d', '2019-07-10'),     1480), 
(strftime('%Y-%m-%d', '2019-07-11'),     1405), 
(strftime('%Y-%m-%d', '2019-07-12'),     1502) 
'''


In [ ]:
cur.executescript(sql)

In [ ]:
sql = '''select t.* from oper t  '''

In [ ]:
select(sql)

,date,cnt
0,2019-06-02,1985
1,2019-06-03,1577
2,2019-06-04,1597
3,2019-06-05,1468
4,2019-07-06,82
5,2019-07-08,1689
6,2019-07-09,1556
7,2019-07-10,1480
8,2019-07-11,1405
9,2019-07-12,1502


###Правильное решение:

In [ ]:
sql = '''
with oper_start_of_month as (
select t.*, date(t.date, 'start of month') as start_of_month
from oper t)


select t.*, 
sum(t.cnt) over (partition by t.start_of_month order by t.date) as cnt_cum
 from oper_start_of_month t
'''

In [ ]:
select(sql)

,date,cnt,start_of_month,cnt_cum
0,2019-06-02,1985,2019-06-01,1985
1,2019-06-03,1577,2019-06-01,3562
2,2019-06-04,1597,2019-06-01,5159
3,2019-06-05,1468,2019-06-01,6627
4,2019-07-06,82,2019-07-01,82
5,2019-07-08,1689,2019-07-01,1771
6,2019-07-09,1556,2019-07-01,3327
7,2019-07-10,1480,2019-07-01,4807
8,2019-07-11,1405,2019-07-01,6212
9,2019-07-12,1502,2019-07-01,7714


###Ответ

Работает!

##SQL3

Даны месячные срезы сегментов клиентов, нужно получить по каждому клиенту периоды действия каждого сегмента. 

####Заливка данных

In [ ]:
sql = '''
drop table if exists segment;

create table segment ([date] date, client_id varchar(6), segment_id int); 

insert into segment values  
('2018-01-31' ,'A11111', 2), 
('2018-02-28' ,'A11111', 2), 
('2018-03-31' ,'A11111', 1), 
('2018-04-30' ,'A11111', 1), 
('2017-11-30' ,'B22222', 1), 
('2017-10-31' ,'B22222', 1), 
('2017-09-30' ,'B22222', 3), 
('2017-09-30' ,'C33333', 1), 
('2017-10-31' ,'C33333', 1) 
'''

In [ ]:
cur.executescript(sql)

In [ ]:
sql = ''' select s.* from segment s order by s.client_id, s.segment_id, s.date'''

###Решение

In [ ]:
select(sql)

,date,client_id,segment_id
0,2018-03-31,A11111,1
1,2018-04-30,A11111,1
2,2018-01-31,A11111,2
3,2018-02-28,A11111,2
4,2017-10-31,B22222,1
5,2017-11-30,B22222,1
6,2017-09-30,B22222,3
7,2017-09-30,C33333,1
8,2017-10-31,C33333,1


Раз нужно указать периоды действия, а мы имеем даты окончания сегмента, нужно для каждой даты найти начало месяца

In [ ]:
sql = '''select 
  s.*, 
  date(s.date, 'start of month') as start_date
from segment s
order by s.client_id, s.segment_id, s.date'''

In [ ]:
select(sql)

,date,client_id,segment_id,start_date
0,2018-03-31,A11111,1,2018-03-01
1,2018-04-30,A11111,1,2018-04-01
2,2018-01-31,A11111,2,2018-01-01
3,2018-02-28,A11111,2,2018-02-01
4,2017-10-31,B22222,1,2017-10-01
5,2017-11-30,B22222,1,2017-11-01
6,2017-09-30,B22222,3,2017-09-01
7,2017-09-30,C33333,1,2017-09-01
8,2017-10-31,C33333,1,2017-10-01


In [ ]:
sql = '''
select 
  s.client_id,
  s.segment_id,
  min(date(s.date,'start of month')) as start_date,
  max(s.date) as end_date
from segment s
group by s.client_id, s.segment_id'''

In [ ]:
select(sql)

,client_id,segment_id,start_date,end_date
0,A11111,1,2018-03-01,2018-04-30
1,A11111,2,2018-01-01,2018-02-28
2,B22222,1,2017-10-01,2017-11-30
3,B22222,3,2017-09-01,2017-09-30
4,C33333,1,2017-09-01,2017-10-31


Проверим:

In [ ]:
sql = '''select s.* from segment s'''

In [ ]:
select(sql)

,date,client_id,segment_id
0,2018-01-31,A11111,2
1,2018-02-28,A11111,2
2,2018-03-31,A11111,1
3,2018-04-30,A11111,1
4,2017-11-30,B22222,1
5,2017-10-31,B22222,1
6,2017-09-30,B22222,3
7,2017-09-30,C33333,1
8,2017-10-31,C33333,1


###Ответ
Все верно, ответ получен

####Еще одно решение

Запишем для каждого сегмента минимальную дату и максимальную, а потом сгруппируем:

In [ ]:
sql = ''' 
with segment_ext as (
select t.*, t.client_id || '_' || t.segment_id as client_id_segment_id from segment t)

  select t.*,
  min(date(t.date,'start of month')) over (partition by t.client_id_segment_id) as date_min, 
  max(t.date) over (partition by t.client_id_segment_id) as date_max
  from segment_ext t
'''

In [ ]:
select(sql)

,date,client_id,segment_id,client_id_segment_id,date_min,date_max
0,2018-03-31,A11111,1,A11111_1,2018-03-01,2018-04-30
1,2018-04-30,A11111,1,A11111_1,2018-03-01,2018-04-30
2,2018-01-31,A11111,2,A11111_2,2018-01-01,2018-02-28
3,2018-02-28,A11111,2,A11111_2,2018-01-01,2018-02-28
4,2017-11-30,B22222,1,B22222_1,2017-10-01,2017-11-30
5,2017-10-31,B22222,1,B22222_1,2017-10-01,2017-11-30
6,2017-09-30,B22222,3,B22222_3,2017-09-01,2017-09-30
7,2017-09-30,C33333,1,C33333_1,2017-09-01,2017-10-31
8,2017-10-31,C33333,1,C33333_1,2017-09-01,2017-10-31
